<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/DEF_v1_BERT_BBDD_FAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langdetect deep_translator faker tqdm transformers torch

import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from faker import Faker
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# Configuración inicial
DetectorFactory.seed = 0
fake_es = Faker("es_ES")

# Cargar modelo BERT para NER en español
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")


def generar_nombre_con_palabras(original_name):
    """Genera un nombre falso con el mismo número de palabras que el original"""
    num_palabras = len(original_name.split())

    # Generar nombres hasta que coincida el número de palabras
    while True:
        if num_palabras == 1:
            nombre = fake_es.first_name()
        elif num_palabras == 2:
            nombre = f"{fake_es.first_name()} {fake_es.last_name()}"
        else:
            nombre = f"{fake_es.first_name()} {fake_es.last_name()} {fake_es.last_name()}"

        if len(nombre.split()) == num_palabras:
            return nombre

def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajuste fino para capturar correctamente los espacios
            while start > 0 and texto[start-1] != ' ':
                start -= 1
            while end < len(texto) and texto[end] != ' ':
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades

def traducir_y_anonimizar(texto):
    """Traduce y anonimiza manteniendo el número de palabras"""
    if not texto.strip():
        return texto

    # Traducción
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Anonimizar nombres en "Me llamo..."
    texto = re.sub(
        r"(Me llamo\s+)([A-ZÁÉÍÓÚÑa-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑa-záéíóúñ]+)*)",
        lambda match: match.group(1) + generar_nombre_con_palabras(match.group(2)),
        texto,
        flags=re.IGNORECASE
    )

    # Anonimizar números
    texto = re.sub(r"\b\d{9}\b", lambda _: fake_es.phone_number(), texto)
    texto = re.sub(r"\b\d{8}[A-Za-z]\b", lambda _: fake_es.ssn(), texto)

    # Detección de entidades con BERT
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre_con_palabras(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto




def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row

# Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")




Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
  0%|          | 0/6943 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length.

Procesamiento completado en 878.13 segundos.


In [1]:
!pip install langdetect deep_translator faker tqdm transformers torch

import sqlite3
import pandas as pd
import re
import time
from langdetect import detect, DetectorFactory, LangDetectException
from deep_translator import GoogleTranslator
from faker import Faker
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from transformers import pipeline

# Configuración inicial
DetectorFactory.seed = 0
fake_es = Faker("es_ES")

# Cargar modelo BERT para NER en español
ner_model = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner", aggregation_strategy="simple")


def detectar_genero(nombre):
    """Detecta si un nombre es masculino o femenino según Faker"""
    if nombre in fake_es.first_name_male():
        return "male"
    elif nombre in fake_es.first_name_female():
        return "female"
    return "neutral"


def generar_nombre_con_palabras(original_name):
    """Genera un nombre falso con el mismo número de palabras y el mismo género que el original"""
    palabras = original_name.split()
    num_palabras = len(palabras)

    # Determinar el género del primer nombre
    genero = detectar_genero(palabras[0])

    while True:
        if num_palabras == 1:
            nombre = fake_es.first_name_male() if genero == "male" else fake_es.first_name_female()
        elif num_palabras == 2:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()}"
        else:
            nombre = f"{fake_es.first_name_male() if genero == 'male' else fake_es.first_name_female()} {fake_es.last_name()} {fake_es.last_name()}"

        if len(nombre.split()) == num_palabras:
            return nombre


def detectar_entidades(texto):
    """Detecta entidades PER usando BERT"""
    resultados = ner_model(texto)
    entidades = []

    for ent in resultados:
        if ent['entity_group'] == 'PER':
            start = ent['start']
            end = ent['end']
            original = texto[start:end]

            # Ajuste fino para capturar correctamente los espacios
            while start > 0 and texto[start-1] != ' ':
                start -= 1
            while end < len(texto) and texto[end] != ' ':
                end += 1

            entidades.append((start, end, original.strip()))

    return entidades


def traducir_y_anonimizar(texto):
    """Traduce y anonimiza manteniendo el número de palabras y el género"""
    if not texto.strip():
        return texto

    # Traducción
    try:
        if len(texto) > 3 and detect(texto) != "es":
            texto = GoogleTranslator(source="auto", target="es").translate(texto)
    except LangDetectException:
        pass

    # Anonimizar nombres en "Me llamo..."
    texto = re.sub(
        r"(Me llamo\s+)([A-ZÁÉÍÓÚÑa-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑa-záéíóúñ]+)*)",
        lambda match: match.group(1) + generar_nombre_con_palabras(match.group(2)),
        texto,
        flags=re.IGNORECASE
    )

    # Anonimizar números
    texto = re.sub(r"\b\d{9}\b", lambda _: fake_es.phone_number(), texto)
    texto = re.sub(r"\b\d{8}[A-Za-z]\b", lambda _: fake_es.ssn(), texto)

    # Detección de entidades con BERT
    entidades = detectar_entidades(texto)
    replacements = []

    for start, end, original in entidades:
        fake_name = generar_nombre_con_palabras(original)
        replacements.append((start, end, fake_name))

    # Aplicar reemplazos en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    return texto





def procesar_fila(row):
    """Procesa una fila aplicando traducción y anonimización."""
    row["body"] = traducir_y_anonimizar(row["body"])
    return row

# Conectar a la base de datos
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    # Crear la tabla si no existe
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS mi_tabla (
            id INTEGER PRIMARY KEY,
            body TEXT,
            secret TEXT,
            direction TEXT,
            createdAt TEXT,
            OpenchannelAccountId INTEGER,
            OpenchannelInteractionId INTEGER,
            UserId INTEGER,
            ContactId INTEGER,
            AttachmentId INTEGER,
            sentBy TEXT
        );
    """)
    conn.commit()

    # Cargar datos desde Excel (solo si es necesario)
    df = pd.read_excel("MOSTRA_1.xlsx")
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer solo columnas relevantes
    df_body = df[["id", "body", "direction","createdAt","UserId", "ContactId"]].copy()

    # Usar tqdm para mostrar progreso
    start_time = time.time()
    with Pool(cpu_count()) as pool:
        result = list(tqdm(pool.imap(procesar_fila, df_body.to_dict(orient="records")), total=len(df_body)))

    # Convertir la lista de diccionarios de vuelta a DataFrame
    df_body = pd.DataFrame(result)

    # Actualizar base de datos en un solo paso eficiente
    df_body.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    elapsed_time = time.time() - start_time
    print(f"Procesamiento completado en {elapsed_time:.2f} segundos.")




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
  0%|          | 0/6943 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 6943/6943 [11:45<00:00,  9.85it/s]


Procesamiento completado en 705.52 segundos.


In [4]:
from transformers import pipeline
import sqlite3
import pandas as pd
import time

# Cargar el modelo de análisis de sentimiento en español
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Conectar a la base de datos
def analizar_sentimientos():
    with sqlite3.connect("mi_base_de_datos.db") as conn:
        df = pd.read_sql_query("SELECT id, body, direction FROM mi_tabla", conn)

        # Función para analizar el sentimiento
        def analizar_sentimiento(text, total, index):
            try:
                resultado = sentiment_pipeline(text)[0]['label']
            except Exception as e:
                print(f"Error en el análisis de sentimiento: {e}")
                resultado = "Neutral"  # Valor por defecto en caso de error

            # Calcular progreso
            porcentaje = (index + 1) / total * 100
            tiempo_restante = (time.time() - inicio) / (index + 1) * (total - (index + 1))
            print(f"Progreso: {porcentaje:.2f}% - Tiempo restante estimado: {tiempo_restante:.2f} segundos")

            return resultado

        # Subset para 'in' y 'out'
        df_in = df[df['direction'] == 'in'].copy()
        df_out = df[df['direction'] == 'out'].copy()

        # Analizar sentimiento con progreso
        total_in = len(df_in)
        total_out = len(df_out)
        global inicio
        inicio = time.time()
        df_in['sentiment'] = [analizar_sentimiento(text, total_in, i) for i, text in enumerate(df_in['body'])]
        df_out['sentiment'] = [analizar_sentimiento(text, total_out, i) for i, text in enumerate(df_out['body'])]

        # Guardar los resultados en nuevas tablas
        df_in.to_sql("mi_tabla_con_sentimiento_in", conn, if_exists="replace", index=False)
        df_out.to_sql("mi_tabla_con_sentimiento_out", conn, if_exists="replace", index=False)

        # Imprimir algunos resultados para verificar
        print("Sentimientos para 'in':")
        print(df_in.head())
        print("Sentimientos para 'out':")
        print(df_out.head())

# Ejecutar el análisis
if __name__ == "__main__":
    analizar_sentimientos()




Device set to use cpu


Se han truncado las últimas 5000 líneas del flujo de salida.
Progreso: 53.85% - Tiempo restante estimado: 158.63 segundos
Progreso: 53.88% - Tiempo restante estimado: 158.51 segundos
Progreso: 53.91% - Tiempo restante estimado: 158.39 segundos
Progreso: 53.93% - Tiempo restante estimado: 158.29 segundos
Progreso: 53.96% - Tiempo restante estimado: 158.18 segundos
Progreso: 53.99% - Tiempo restante estimado: 158.09 segundos
Progreso: 54.01% - Tiempo restante estimado: 157.98 segundos
Progreso: 54.04% - Tiempo restante estimado: 157.86 segundos
Progreso: 54.07% - Tiempo restante estimado: 157.85 segundos
Progreso: 54.10% - Tiempo restante estimado: 157.79 segundos
Progreso: 54.12% - Tiempo restante estimado: 157.70 segundos
Progreso: 54.15% - Tiempo restante estimado: 157.60 segundos
Progreso: 54.18% - Tiempo restante estimado: 157.50 segundos
Progreso: 54.21% - Tiempo restante estimado: 157.41 segundos
Progreso: 54.23% - Tiempo restante estimado: 157.32 segundos
Progreso: 54.26% - Tiemp

In [5]:
# prompt: hacer lista con las frases mas positivas y mas negativas de cada dataset

import sqlite3
import pandas as pd

def obtener_frases_extremas():
    with sqlite3.connect("mi_base_de_datos.db") as conn:
        # Obtener datos de las tablas con sentimiento
        df_in = pd.read_sql_query("SELECT body, sentiment FROM mi_tabla_con_sentimiento_in", conn)
        df_out = pd.read_sql_query("SELECT body, sentiment FROM mi_tabla_con_sentimiento_out", conn)

        # Obtener las frases más positivas y negativas para 'in'
        positivas_in = df_in[df_in['sentiment'] == '5 stars'].sort_values(by='body').head(5)['body'].tolist()  # Ajusta el número de frases según sea necesario
        negativas_in = df_in[df_in['sentiment'] == '1 star'].sort_values(by='body').head(5)['body'].tolist()


        # Obtener las frases más positivas y negativas para 'out'
        positivas_out = df_out[df_out['sentiment'] == '5 stars'].sort_values(by='body').head(5)['body'].tolist()
        negativas_out = df_out[df_out['sentiment'] == '1 star'].sort_values(by='body').head(5)['body'].tolist()

        print("Frases más positivas (in):")
        for frase in positivas_in:
            print(frase)

        print("\nFrases más negativas (in):")
        for frase in negativas_in:
            print(frase)

        print("\nFrases más positivas (out):")
        for frase in positivas_out:
            print(frase)

        print("\nFrases más negativas (out):")
        for frase in negativas_out:
            print(frase)


# Ejecutar la función
if __name__ == "__main__":
    obtener_frases_extremas()


Frases más positivas (in):
!
12 años llevo de encargado alli
15
17 años
20

Frases más negativas (in):
*aún no he acabado
01:10
1. Literalmente una profesora me dijo que esto no era la facultad de psicología y que no les importan nuestros sentimientos
2. Sólo al principio, pero me he dado cuenta de que mi verdadera pasión es la historia
3. No, ya voy con inseguridades desde segundo. Pero como soy como soy, me negué a dejarla y ahora estoy pagando las consecuencias
3 meses
3 meses

Frases más positivas (out):
*Aquí lo escuchamos*
Es un servicio en el que puede hablar sobre cómo se siente, resolver conflictos, trabajar en autoejes ..., es individual y confidencial. 


https://ajuntament.barcelona.cat/joves/ca/canal/aqui-tecoltem
📞+34845954455
*Aquí lo escuchamos*
Es un servicio en el que puede hablar sobre cómo se siente, resolver conflictos, trabajar en autoejes ..., es individual y confidencial. 

https://ajuntament.barcelona.cat/joves/ca/canal/aqui-tecoltem

📞+34885 39 28 00
*Aquí lo 